In [51]:
import onnx
from onnx import helper,TensorProto
from finn.core.modelwrapper import ModelWrapper
from finn.core.datatype import DataType
import numpy as np
from finn.util.basic import gen_finn_dt_tensor
import onnx.version_converter as vc
from onnx.backend.test.case.node import expect

model = ModelWrapper("tinyyolo-20210831.onnx")
model.get_initializer("737")


array([0])

In [102]:
# shape node 
top_in = helper.make_tensor_value_info("top_in", TensorProto.FLOAT, [1,64,16,16])
shape_out = helper.make_tensor_value_info("735", TensorProto.INT64,[4])
shape_node = helper.make_node(
            "Shape",
            inputs = ["top_in"],
            outputs = ["735"])

In [103]:
# Slice node
input_shape = [4]
output_shape = [2]
param_shape = [1]
idt = DataType["UINT8"]
param_dt = DataType["FLOAT32"]
slice_in = helper.make_tensor_value_info("735", TensorProto.INT64, input_shape)
slice_out = helper.make_tensor_value_info("slice_out", TensorProto.INT64, output_shape)
slice_attr = {}
# slice_attr["starts"] = [1]
# slice_attr["ends"] = [1]
# slice_attr["axes"] = [1]

value_info = [
        helper.make_tensor_value_info("starts", TensorProto.INT8, param_shape),
        helper.make_tensor_value_info("ends",   TensorProto.INT8, param_shape),
        helper.make_tensor_value_info("axes",   TensorProto.INT8, param_shape),
]

slice_node = helper.make_node(
            "Slice",
            inputs  = ["735","starts","ends","axes"],
            outputs = ["slice_out"],
            **slice_attr
            )


In [108]:
# Concat
concat_in = helper.make_tensor_value_info("slice_out", TensorProto.INT64, [2])
p1 = helper.make_tensor_value_info("839", TensorProto.INT64,[2])
top_out = helper.make_tensor_value_info("top_out", TensorProto.INT64,[4])
concat_config = {}
concat_config["axis"] = np.int64(0)
concat_node = helper.make_node(
            "Concat",
            inputs = ["slice_out","839"],
            outputs = ["top_out"],
            **concat_config)

# concat_model = helper.make_model(
    
#     helper.make_graph(    
#         [concat_node],
#         inputs=[concat_in],
#         outputs = [top_out],
#         name = "concat_graph",
#         value_info=[p1])
# )

# temp_model = ModelWrapper(concat_model)
# temp_model.set_initializer("839",np.array([10,10]))

# temp_model = temp_model.set_initializer("839",np.array([20,20]))
# temp_model.save("Concat.onnx")

In [109]:
graph = helper.make_graph(
    nodes = [shape_node,slice_node,concat_node],
    name = "slice_graph",
    inputs = [top_in],
    outputs = [top_out],
    value_info = value_info,
)

In [110]:
model_config = {}
model_config["opset_imports"] = [helper.make_operatorsetid("",11)]

modelProto = helper.make_model(graph,**model_config)
model = ModelWrapper(modelProto)

# model.set_tensor_datatype("starts", param_dt)
# model.set_initializer("starts",gen_finn_dt_tensor(param_dt,param_shape))
# model.set_tensor_datatype("ends", param_dt)
# model.set_initializer("ends",gen_finn_dt_tensor(param_dt,param_shape))

model.set_initializer("starts",np.array([0]))
model.set_initializer("ends",np.array([0]))
model.set_initializer("axes",np.array([0]))
model.set_initializer("839",np.array([10,10]))


# model.set_tensor_datatype("axes", param_dt)
# model.set_initializer("axes",gen_finn_dt_tensor(param_dt,param_shape))

# converted_model = vc.convert_version(model.model,12)
# model = ModelWrapper(converted_model)
model.save("slice.onnx")

In [111]:
import finn.builder.build_dataflow as build
import finn.builder.build_dataflow_config as build_cfg
import os
import shutil

model_file = "slice.onnx"

final_output_dir = "output_files/slice"

#Delete previous run results if exist
if os.path.exists(final_output_dir):
    shutil.rmtree(final_output_dir)
    print("Previous run results deleted!")

cfg = build.DataflowBuildConfig(
    output_dir          = final_output_dir,
    mvau_wwidth_max     = 80,
    target_fps          = 1000000,
    synth_clk_period_ns = 10.0,
    board               = "KV260_SOM",
#     steps               = "estimate_only_dataflow_steps",
    shell_flow_type     = build_cfg.ShellFlowType.VIVADO_ZYNQ,
    generate_outputs=[
        build_cfg.DataflowOutputType.BITFILE,
        build_cfg.DataflowOutputType.PYNQ_DRIVER,
        build_cfg.DataflowOutputType.DEPLOYMENT_PACKAGE,
    ]
)

In [112]:
%%time
build.build_dataflow_cfg(model_file, cfg)

Traceback (most recent call last):
  File "/workspace/finn/src/finn/builder/build_dataflow.py", line 166, in build_dataflow_cfg
    model = transform_step(model, cfg)
  File "/workspace/finn/src/finn/builder/build_dataflow_steps.py", line 195, in step_tidy_up
    model = model.transform(InferShapes())
  File "/workspace/finn-base/src/finn/core/modelwrapper.py", line 141, in transform
    (transformed_model, model_was_changed) = transformation.apply(
  File "/workspace/finn-base/src/finn/transformation/infer_shapes.py", line 88, in apply
    model = ModelWrapper(si.infer_shapes(model.model))
  File "/opt/conda/lib/python3.8/site-packages/onnx/shape_inference.py", line 35, in infer_shapes
    inferred_model_str = C.infer_shapes(model_str, check_type)
RuntimeError: Inferred shape and existing shape differ in dimension 0: (2) vs (4)


Building dataflow accelerator from slice.onnx
Intermediate outputs will be generated in /workspace/results
Final outputs will be generated in output_files/slice
Build log is at output_files/slice/build_dataflow.log
Running step: step_tidy_up [1/16]
> /opt/conda/lib/python3.8/site-packages/onnx/shape_inference.py(35)infer_shapes()
     32         raise TypeError('Shape inference only accepts ModelProto, '
     33                          'incorrect type: {}'.format(type(model)))
     34     model_str = model.SerializeToString()
---> 35     inferred_model_str = C.infer_shapes(model_str, check_type)
     36     return onnx.load_from_string(inferred_model_str)

--KeyboardInterrupt--

KeyboardInterrupt: Interrupted by user
Build failed
CPU times: user 184 ms, sys: 43.6 ms, total: 227 ms
Wall time: 13 s


-1